In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score


In [ ]:
Data = pd.read_csv('Datasets/Clean_data/Locality_Analyst.csv')

In [3]:
Data.head()

,Vùng,Năm,Diện tích(Km2),Dân số trung bình (Nghìn người),Mật độ dân số (Người/km2),Tổng dân số,Nam,Nữ,Tổng dân số thành thị,Tổng dân số nông thôn,...,Tỷ suất chết thô,Tỷ lệ tăng tự nhiên,Tỷ suất nhập cư,Tỷ suất xuất cư,Tỷ suất di cư thuần,Tỷ lệ nam nữ,Khu vực,Tỉ lệ thất nghiệp,Tỉ lệ thất nghiệp ở thành thị,Tỉ lệ thất nghiệp ở nông thôn
0,Hà Nội,2011,3328.9,6825.8,2050.5,6825.82,3359.81,3466.00,2900.37,3925.45,...,6.8,11.8,11.0,6.4,4.7,96.94,Đồng bằng sông Hồng,3.35,2.82,1.48
1,Hà Nội,2012,3323.6,6991.4,2103.6,6991.43,3444.43,3547.01,2972.61,4018.82,...,7.2,9.9,6.1,3.3,2.8,97.11,Đồng bằng sông Hồng,1.19,1.34,5.59
2,Hà Nội,2013,3324.3,7128.4,2144.3,7128.39,3515.06,3613.33,3024.64,4103.75,...,7.3,9.2,7.7,7.4,0.3,97.28,Đồng bằng sông Hồng,1.92,0.89,5.15
3,Hà Nội,2014,3324.5,7285.5,2191.5,7285.53,3595.78,3689.76,3583.51,3702.03,...,6.6,12.3,7.5,7.8,-0.4,97.45,Đồng bằng sông Hồng,1.60,2.07,4.65
4,Hà Nội,2015,3324.5,7433.6,2236.0,7433.60,3672.20,3761.40,3650.50,3783.10,...,7.3,9.1,4.7,4.1,0.6,97.60,Đồng bằng sông Hồng,4.13,3.77,3.63


In [4]:
from sklearn.base import BaseEstimator, TransformerMixin

class CombineAttributesAdder(BaseEstimator,TransformerMixin):

    def fit(self,X,y=None):
        X = X.copy()
        return self
    
    def transform(self,X):
        X = X.copy()
        X = X.sort_values(['Vùng',"Năm"])
        X['prev_density'] = X.groupby('Vùng')['Mật độ dân số (Người/km2)'].shift(1)
        X['growth_density'] = X.groupby('Vùng')['Mật độ dân số (Người/km2)'].pct_change()

        return X.reset_index(drop=True)
    
class StrEncoding (BaseEstimator,TransformerMixin):
    def __init__(self):
        self.le_region = LabelEncoder()
        self.le_area = LabelEncoder()

    def fit(self,X,y=None):
        X = X.copy()
        self.le_region.fit(X['Vùng'])
        self.le_area.fit(X['Khu vực'])
        return self

    def transform(self,X):
        X = X.copy()
        X['Vùng'] = self.le_region.fit_transform(X['Vùng'])
        X['Khu vực'] = self.le_area.fit_transform(X['Khu vực'])
        return X
    

In [5]:
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('Labelencode',StrEncoding()),
    ('attribs_adder',CombineAttributesAdder())
])

In [26]:
class XGBModel:
    def __init__(self,region_name,features):
        self.region = region_name
        self.features = features
        self.XGBmodel = XGBRegressor(
            n_estimators = 500,
            learning_rate = 0.05,
            max_depth = 6,
            subsample = 0.8,
            colsample_bytree = 0.8,
            random_state = 42
        )

    def get_data(self):
        if self.region == 'All': return num_pipeline.fit_transform(Data)
        return num_pipeline.fit_transform(Data[Data['Vùng'] == self.region])
    
    def data_splits(self,data):
        train = data[data['Năm'] <= 2021]
        test = data[data['Năm'] > 2021]

        return train[self.features], train['Tổng dân số'], test[self.features], test['Tổng dân số']
    

    def train_XGBoost_model(self):
        df = self.get_data()
        X_train,y_train, X_test,y_test = self.data_splits(df)
        self.XGBmodel.fit(X_train,y_train)
        y_pred = self.XGBmodel.predict(X_test)

        return pd.DataFrame(np.column_stack([y_pred,y_test]),columns=['y_pred','y_test']).to_csv(f'Result/Test_set/XGBModel/{self.region}_predict_result.csv')

    def CrossValScore(self,scoring):
        X = num_pipeline.fit_transform(Data)
        prepared = X[self.features]
        label = X['Tổng dân số']
        return cross_val_score(self.XGBmodel,prepared,label,scoring=scoring,cv=5)    

In [57]:
features = ['Vùng','Khu vực','Năm','Tỷ lệ nam nữ', 'Nam', 'Nữ',
       'Tổng dân số thành thị', 'Tổng dân số nông thôn', 'Diện tích(Km2)',
       'Dân số trung bình (Nghìn người)', 'Mật độ dân số (Người/km2)',
       'Tỷ suất sinh thô', 'Tỷ suất chết thô', 'Tỷ lệ tăng tự nhiên',
       'Tỷ suất nhập cư', 'Tỷ suất xuất cư', 'Tỷ suất di cư thuần',
       'Tỉ lệ thất nghiệp', 'Tỉ lệ thất nghiệp ở thành thị',
       'Tỉ lệ thất nghiệp ở nông thôn','prev_density','growth_density']


In [29]:
PredictModel = XGBModel('Hà Nội',features)

In [30]:
PredictModel.train_XGBoost_model()
PredictModel.CrossValScore('r2')

array([0.95171148, 0.99659786, 0.99836313, 0.9069687 , 0.99897874])